## Loading Modules

In [1]:
import os
import numpy as np
import pandas as pd

from utils import *

from synapticTrack.io import BeamDataIOManager
from synapticTrack.lattice.track_elements import *
from synapticTrack.lattice.track_lattice import Lattice
from synapticTrack.track.run_track import *

## Run Track

In [2]:
import time
t0 = time.time()

# Load LEBT lattice
elements_list, stms = load_LEBT_lattice()

# ouput files and directories
output_dir_list = ['run.wire_scanner1', 'run.allison_scanner', 'run.wire_scanner2', 'run.wire_scanner3', 'run.wire_scanner4', 'run.ends']

final_output_files = ['coord_wire_scanner1.out', 'coord_allison_scanner.out', 'coord_wire_scanner2.out', 'coord_wire_scanner3.out', 
                      'coord_wire_scanner4.out', 'coord_ends.out']

final_output_dir = "run.00"
os.makedirs(final_output_dir, exist_ok=True)

# Original kick strengths of steering magnets
fh = [5.0, 5.0, 5.0, 5.0, 5.0]
fv = [5.0, 5.0, 5.0, 5.0, 5.0]
fhkick = [0.0, 0.0, 0.0, 0.0, 0.0]
fvkick = [0.0, 0.0, 0.0, 0.0, 0.0]

# Load steering magnet scaling factors
stm_scale_x = np.load("stm_scale_x.npy")
stm_scale_y = np.load("stm_scale_y.npy")

# Apply scaling factors to kick strengths
fhkick *= stm_scale_x
fvkick *= stm_scale_y

# Apply new kick strengths to steering magnets
set_stm_strengths(stms, fhkick, fvkick)

# Run track simulations
run_track_steps(elements_list, output_dir_list)

# Copy output files
copy_step_output_files(final_output_dir, output_dir_list, final_output_files)
copy_track_output_files(final_output_dir, output_dir_list)

# Compute centroids
centroid, rms = get_beam_data(final_output_dir, final_output_files)
x_center = centroid['x'].apply(lambda x: float(f"{x: 11.8f}")).tolist()
y_center = centroid['y'].apply(lambda x: float(f"{x: 11.8f}")).tolist()

# Load wire scanner offsets
ws_offset_x = np.load("ws_offset_x.npy")
ws_offset_y = np.load("ws_offset_y.npy")

# Apply beam centroid offsets at wire scanners
for l, ws in enumerate([0, 2, 3, 4]):
    x_center[ws] = x_center[ws] - ws_offset_x[l]
    y_center[ws] = y_center[ws] - ws_offset_y[l]
        
t1 = time.time()
print (t1 - t0)

14.740877866744995


### Scanner location calculations

In [3]:
z_scanner = []
lattice_length = 0
for elements, output_dir in zip(elements_list, output_dir_list):
    sclinac = Lattice(elements)
    #print('\n')
    for elem in sclinac:
        elem_name = elem.get_name()
        elem_length = elem.get_length()
        lattice_length += elem_length
        #print (f"{elem_name}, {elem_length:3.6f}, {lattice_length*0.01:6.6f}")
    z_scanner.append(lattice_length)

In [4]:
with open(os.path.join(final_output_dir, 'z_scanner.out'), 'w') as f:
    for item in z_scanner:
        f.write(str(item) + '\n')

In [5]:
with open(os.path.join(final_output_dir, 'new_step.out'), 'w') as f:
    for z, x, y in zip(z_scanner, x_center, y_center):
        f.write(f"{z} {x} {y}\n")